In [74]:
import json
prompt_dict = json.load(open("./final_prompt.json"))
prompt_list = list(prompt_dict.values())
question = list(prompt_dict.keys())
print(prompt_dict[question[0]])

================================= Entity Chunks for VEHICLE-TO-GRID =================================
Entity: VEHICLE-TO-GRID
Description: The requirement V2G3-M06-14 is related to the V2G process as it specifies conditions for initiating the V2G charging loop The EV is involved in the V2G process, which includes charging and communication with the grid HLC-C is involved in the V2G setup, facilitating communication between the vehicle and the grid V2G is a set of standards that falls under the ISO 15118 specifications EVCC is a component involved in V2G technology, facilitating communication between the electric vehicle and the grid SECC is a component involved in V2G technology, facilitating communication between the charging station and the electric vehicle PICS_CMN_CMN_CombinedTesting involves V2G messaging as part of the testing The test case TC_SECC_AC_VTB_PLCLinkStatus_003 is designed to test scenarios within the V2G communication framework SessionStopReq is a protocol message us

In [88]:
from llmlingua import PromptCompressor
llm_lingua = PromptCompressor("microsoft/phi-2")
try:
    with open("./compressed_prompt.json", "r") as f:
        compressed_prompt_dict = json.load(f)
    print(f"Loaded {len(compressed_prompt_dict)} existing compressed prompts")
except FileNotFoundError:
    compressed_prompt_dict = {}

for question, prompt in prompt_dict.items():
    compressed_prompt = llm_lingua.compress_prompt(
        prompt,
        question=question,
        condition_in_question="after_condition",
        rate=0.80,
        reorder_context="sort",
        dynamic_context_compression_ratio=0.3, # or 0.4
        condition_compare=True,
        context_budget="*1.5",
        rank_method="longllmlingua",
    )
    compressed_prompt_dict[question] = compressed_prompt
    
    # Save each compressed prompt as we generate it
    with open("./compressed_prompt.json", "w") as f:
        json.dump(compressed_prompt_dict, f, indent=2)
        print(f"Saved compressed prompt for question: {question}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (6870 > 2048). Running this sequence through the model will result in indexing errors


Loaded 0 existing compressed prompts
Saved compressed prompt for question: What is the system architecture of Vehicle-to-Grid?
Saved compressed prompt for question: What is HomePlug Green PHY?
Saved compressed prompt for question: What is Signal Level Attenuation Characterization?
Saved compressed prompt for question: What is Logical Network?
Saved compressed prompt for question: What is Central Coordinator?
Saved compressed prompt for question: What is the difference between Service Access Point for Data and Data link control SAP?
Saved compressed prompt for question: What is the difference between basic signaling and high-level communication?
Saved compressed prompt for question: What is the difference between Main Test Component and Parallel Test Component?
Saved compressed prompt for question: What is ABSTRACT TEST SUITE?
Saved compressed prompt for question: What is TEST SUITE STRUCTURE?
Saved compressed prompt for question: What is External Identification Means?
Saved compressed 

In [ ]:
json.dump(compressed_prompt_dict, open("./compressed_prompt.json", "w"))

In [57]:
from llmlingua import PromptCompressor
llm_lingua = PromptCompressor("microsoft/phi-2")
def compress_prompt(question, prompt, compress_ratio):
    res = llm_lingua.compress_prompt(
                prompt,
                # question=question,
                # condition_in_question="after_condition",
                rate=compress_ratio,
                # reorder_context="sort",
                # dynamic_context_compression_ratio=0.3, # or 0.4
                # condition_compare=True,
                # context_budget="+100",
                # rank_method="longllmlingua",
            )
    # compressed_prompt = "\n".join(res['compressed_prompt'].split('\n')[:-1])
    compressed_prompt = res['compressed_prompt']
    origin_tokens = res['origin_tokens']
    compressed_tokens = res['compressed_tokens']
    rate = res['rate']
    return [compressed_prompt, origin_tokens, compressed_tokens, rate]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [70]:
def get_common_entities_dict(data, max_distance=-1):
    """
    Returns a new dictionary containing only the common entities across all chunks,
    maintaining the original structure for each entity
    Args:
        data: dictionary containing entity_chunk_X keys with entity information
    Returns:
        dict: new dictionary with only common entities
    """
    # First find common entity names across all chunks
    chunk_keys = list(data.keys())
    if not chunk_keys:
        return {}
    
    # Get common entity names (like 'EIM', 'AC', etc.)
    common_entities = set(data[chunk_keys[0]].keys())
    for chunk_key in chunk_keys[1:]:
        common_entities = common_entities.intersection(set(data[chunk_key].keys()))
    
    # Create new dictionary with only common entities
    common_dict = {}
    
    # First handle main entities (distance=0)
    for chunk_key in chunk_keys:
        main_entity = next((entity for entity, info in data[chunk_key].items() 
                        if info.get('distance') == 0), None)
        
        if main_entity and (max_distance < 0 or 0 <= max_distance):
            if main_entity not in common_dict:
                common_dict[main_entity] = data[chunk_key][main_entity]
    
    # Then handle other common entities
    for entity in common_entities:
        if entity in common_dict:  # Skip if already added as main entity
            continue
            
        # Find smallest distance for this entity across all chunks
        smallest_distance = float('inf')
        best_data = None
        
        for chunk_key in chunk_keys:
            entity_data = data[chunk_key].get(entity, {})
            distance = entity_data.get('distance', float('inf'))
            if distance < smallest_distance:
                smallest_distance = distance
                best_data = entity_data
        
        # Only include entity if its smallest distance is within limit
        if max_distance < 0 or smallest_distance <= max_distance:
            common_dict[entity] = best_data
    common_dict = {"common_entities": common_dict}
    return common_dict

def get_unique_entities_dict(data, max_distance=-1):
    """
    Returns a new dictionary containing only the unique entities for each chunk,
    maintaining the original structure for each entity
    Args:
        data: dictionary containing entity_chunk_X keys with entity information
    Returns:
        dict: new dictionary with only unique entities per chunk
    """
    chunk_keys = list(data.keys())
    unique_dict = {}
    # For each chunk, find entities that don't appear in any other chunk
    for current_chunk in chunk_keys:
        current_entities = set(data[current_chunk].keys())
        
        # Get entities from all other chunks
        other_entities = set()
        for other_chunk in chunk_keys:
            if other_chunk != current_chunk:
                other_entities.update(data[other_chunk].keys())
        
        # Get unique entities for this chunk
        unique_entities = current_entities - other_entities
        
        main_entity = next((entity for entity, info in data[current_chunk].items() 
                        if info.get('distance') == 0), None)
        
        # Create new dictionary with unique entities and main entity for this chunk
        unique_dict[current_chunk] = {}
        
        if main_entity:
            unique_dict[current_chunk][main_entity] = data[current_chunk][main_entity]
            
        # Add other unique entities
        for entity in unique_entities:
            if entity != main_entity:  # Don't add main entity twice
                entity_data = data[current_chunk][entity]
                distance = entity_data.get('distance', float('inf'))
                if max_distance < 0 or distance <= max_distance:
                    unique_dict[current_chunk][entity] = entity_data
    
    return unique_dict

def combine_entity_descriptions(data, max_distance=3, entities_chunks_file=None, question=""):
    """
    Combines the 'relationship', 'description', and 'chunk_context' fields
    for each entity in the JSON file into a single detailed description.

    Args:
        data (dict): Dictionary containing entity information
        max_distance (int): Maximum distance to include entities
        
    Returns:
        dict: A dictionary with entity names as keys and their combined descriptions as values.
    """
    entities_chunks = {}
    print(entities_chunks_file, question)
    # Process entities up to max_distance
    for entity_chunk, entities in data.items():
        first_one = True
        main_entity = ""
        for entity, details in entities.items():
            if details.get('distance') == 0:
                if not first_one:
                    main_entity += " & "
                main_entity += entity
                first_one = False
            else: break
        entities_chunks[main_entity] = {}
        
        # Process entities up to max_distance
        for entity, details in entities.items():
            dis = details.get('distance')
            if dis > max_distance:
                continue
            
            combined_parts = []
            
            # Extract and append 'relationship' if it's not empty
            relationship = details.get('relationship', '').strip()
            if relationship:
                combined_parts.append(relationship.strip('"'))  # Remove surrounding quotes if present

            # Extract and append 'description' if it's not empty
            description = details.get('description', '').strip()
            if description:
                # Replace <SEP> with a space or another separator if desired
                description = description.replace('<SEP>', ' ')
                combined_parts.append(description.strip('"'))  # Remove surrounding quotes if present

            # Extract and append 'chunk_context' if it's not empty
            chunk_context = details.get('chunk_context', '').strip()
            if chunk_context:
                combined_parts.append(chunk_context)
            result = ["", 0, 0, 1.00]
            # Join all parts with a space
            combined_description = ' '.join(combined_parts)
            if dis == 0:
                if entities_chunks_file == './fixed_entities_chunks.json':
                    result = compress_prompt(question, combined_description, 0.7)
                else:
                    result = [combined_description, 0, 0, 1.00]
            elif dis == 1:
                if entities_chunks_file == './fixed_entities_chunks.json':
                    result = compress_prompt(question, combined_description, 0.7)
                else:
                    result = compress_prompt(question, combined_description, 0.8)
            elif dis == 2:
                if entities_chunks_file == './fixed_entities_chunks.json':
                    result = compress_prompt(question, combined_description, 0.7)
                else:
                    result = compress_prompt(question, combined_description, 0.6)

            compressed_description = result[0]
            # print(entity, result[3])
            compression_stats = {
                "entity": entity,
                "distance": dis,
                "original_length": len(combined_description),
                "compressed_length": len(compressed_description),
                "compression_ratio": result[3],
                "tokens_before": result[1],
                "tokens_after": result[2],
                "original_prompt": combined_description,
                "compressed_prompt": compressed_description
            }
            
            # Load existing stats
            try:
                with open("/home/Lewis/Delta_Research/experiments/Compression/compression_stats.json", "r") as f:
                    existing_stats = json.load(f)
            except (FileNotFoundError, json.JSONDecodeError):
                # File doesn't exist yet or is empty/corrupted
                existing_stats = []
            
            # Append new stats
            existing_stats.append(compression_stats)
            
            # Write all stats back to file as a proper JSON array
            with open("/home/Lewis/Delta_Research/experiments/Compression/compression_stats.json", "w") as f:
                json.dump(existing_stats, f, indent=4)
            entities_chunks[main_entity][entity] = compressed_description

    return entities_chunks

def format_entity_chunks_prompt(data, main_entity):
    chunks_prompt = f"================================= Entity Chunks for {main_entity} =================================\n"
    for entity, description in data.items():
        chunks_prompt += f"Entity: {entity}\nDescription: {description}\n{'-'*80}\n"
    return chunks_prompt

def extend_entities_from_graph(data, max_distance=1):
    """
    Extends the entities in the data by finding their neighbors in the entity graph
    up to a specified distance.

    Args:
        data (dict): Dictionary containing entity_chunk_X keys with entity information
        max_distance (int): Maximum distance to extend entities in the graph
        
    Returns:
        dict: The original data with additional entities added from the graph
    """
    try:
        # Load the entity graph
        entity_graph = json.load(open("/home/Lewis/Delta_Research/optimized_entity_graph.json"))
        if not entity_graph:
            print("Warning: Could not load entity graph, returning original data")
            return data
            
        # Create a deep copy to avoid modifying the original data
        import copy
        extended_data = copy.deepcopy(data)
        
        # For each chunk in the data
        for chunk_key, entities in extended_data.items():
            # Find all entities at the boundary (maximum distance in current data)
            current_max_distance = max([details.get('distance', 0) for entity, details in entities.items()])
            boundary_entities = [entity for entity, details in entities.items() 
                                if details.get('distance') == current_max_distance]
            
            # Track processed entities to avoid duplicates
            processed_entities = set(entities.keys())
            
            # For each entity at the boundary
            for boundary_entity in boundary_entities:
                _extend_entity_neighbors(
                    entity_graph, 
                    extended_data[chunk_key], 
                    boundary_entity, 
                    current_max_distance,
                    processed_entities, 
                    max_depth=max_distance
                )
                
        return extended_data
        
    except Exception as e:
        print(f"Error extending entities from graph: {e}")
        return data
        
def _extend_entity_neighbors(entity_graph, entities_dict, entity_name, current_distance, 
                            processed_entities, max_depth=1, current_depth=0):
    """
    Recursively extends entity neighbors from the graph.
    
    Args:
        entity_graph (dict): The loaded entity graph
        entities_dict (dict): The dictionary to add new entities to
        entity_name (str): The current entity to find neighbors for
        current_distance (int): The current distance in the original data
        processed_entities (set): Set of already processed entity names
        max_depth (int): Maximum recursion depth
        current_depth (int): Current recursion depth
    """
    # Base case: reached max depth or entity not in graph
    if current_depth >= max_depth or entity_name not in entity_graph:
        return
        
    # Get connections for this entity
    connections = entity_graph.get(entity_name, {}).get("connections", [])
    
    # Add each connection as a new entity if not already processed
    for connection in connections:
        neighbor_name = connection.get("target", "")
        if not neighbor_name or neighbor_name in processed_entities:
            continue
            
        # Mark as processed
        processed_entities.add(neighbor_name)
        
        # Create new entity details
        new_distance = current_distance + 1
        connection_desc = connection.get("description", "")
        
        # Get entity's own description if available
        neighbor_desc = ""
        if neighbor_name in entity_graph:
            neighbor_desc = entity_graph[neighbor_name].get("description", "")
            
        # Create relationship description
        relationship = f"Related to {entity_name}"
        if connection_desc:
            relationship = connection_desc
            
        # Add the new entity
        entities_dict[neighbor_name] = {
            'distance': new_distance,
            'relationship': relationship,
            'description': neighbor_desc,
            'chunk_context': ""  # No context for graph-derived entities
        }
        
        # Recursively extend from this new entity
        _extend_entity_neighbors(
            entity_graph, 
            entities_dict, 
            neighbor_name, 
            new_distance,
            processed_entities, 
            max_depth, 
            current_depth + 1
        )

def generate_final_prompt(data, question: str, intention_questions: str, entities_chunks_file: str):
    final_prompt = ""
    extracted_dis = 1
    # For General Information Query, no extension needed
    if question not in intention_questions:
        # No extension for general queries
        combined_dict = combine_entity_descriptions(data, max_distance=extracted_dis, entities_chunks_file=entities_chunks_file, question=question)
        for query_entity, entity_chunks in combined_dict.items():
            final_prompt += format_entity_chunks_prompt(entity_chunks, query_entity)

    else:
        intention_category = "Comparison Query"
        # Determine which aspect needs extension based on query type
        common_extend = 1 if intention_category == "Commonality Query" else 0
        diff_extend = 1 if intention_category == "Comparison Query" else 0
        
        common_dis = extracted_dis + 1 if intention_category == "Commonality Query" else extracted_dis
        dif_dis = extracted_dis + 1 if intention_category == "Comparison Query" else extracted_dis
        
        print("These are common & diff distance: ", common_dis, dif_dis)
        
        # First extend the entities if needed
        extended_data = data
        if common_extend > 0 or diff_extend > 0:
            extension_distance = max(common_extend, diff_extend)
            extended_data = extend_entities_from_graph(data, max_distance=extension_distance)
        
        # Then extract common/unique entities from the extended data
        common_entities_dict = get_common_entities_dict(extended_data, common_dis) if common_dis != -1 else {}
        common_chunks = combine_entity_descriptions(common_entities_dict, max_distance=common_dis, entities_chunks_file=entities_chunks_file, question=question)
        common_prompts = {}
        num_of_entities = 0
        
        for entity_chunk, chunks in common_chunks.items():
            num_of_entities += len(chunks.keys())
            common_prompts[entity_chunk] = format_entity_chunks_prompt(chunks, entity_chunk)
            
        unique_entities_dict = get_unique_entities_dict(extended_data, dif_dis) if dif_dis != -1 else {}
        unique_chunks = combine_entity_descriptions(unique_entities_dict, max_distance=dif_dis, entities_chunks_file=entities_chunks_file, question=question)
        unique_prompts = {}
        
        for entity_chunk, chunks in unique_chunks.items():
            num_of_entities += len(chunks.keys())
            unique_prompts[entity_chunk] = format_entity_chunks_prompt(chunks, entity_chunk)
            
        for main_entity, chunks in unique_prompts.items():
            final_prompt += f"Below is the unique entity information for {main_entity}\n"
            final_prompt += chunks
        for main_entity, chunks in common_prompts.items():
            final_prompt += f"Below is the common entity information for {main_entity}\n"
            final_prompt += chunks

    final_prompt += f"You need to answer the following question as more details as possible based on the provided information above\n Question: {question}"
    # print(final_prompt)
    # Save the prompt in a dictionary format with the question as the key
    # try:
    #     with open("final_prompt.json", "r") as f:
    #         prompt_dict = json.load(f)
    # except (FileNotFoundError, json.JSONDecodeError):
    #     prompt_dict = {}
    # prompt_dict[question] = final_prompt
    # with open("final_prompt.json", "w") as f:
    #     json.dump(prompt_dict, f, indent=4)
    return final_prompt

In [71]:
import json
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
entities_chunks_files = ['./fixed_entities_chunks.json', './dynamic_entities_chunks.json']
dif_questions = ["What is the difference between Data SAP and Data link control SAP?",
            "What is the difference between basic signaling and high-level communication?",
            "What is the difference between MTC and PTC?"
    ]
for entities_chunks_file in entities_chunks_files:
    entities_chunks = json.load(open(entities_chunks_file))
    output_file = f"{entities_chunks_file.split('.')[1][1:]}_qa.json"
    # print(entities_chunks_file, output_file)
    # Try to load existing QA data
    try:
        with open(output_file, 'r') as f:
            qa_dict = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        qa_dict = {}
    
    for question, data in entities_chunks.items():
        # Skip if question has already been processed
        if question in qa_dict:
            print(f"Skipping already processed question: {question}")
            continue
            
        prompt = generate_final_prompt(data, question, dif_questions, entities_chunks_file)
        response = llm.invoke(prompt)
        qa_dict[question] = response.content
        
        # Save after each question to preserve progress
        with open(output_file, "w") as f:
            json.dump(qa_dict, f, indent=4)
    

Skipping already processed question: What is the system architecture of Vehicle-to-Grid?
Skipping already processed question: What is HPGP?
Skipping already processed question: What is SLAC?
Skipping already processed question: What is Logical Network?
Skipping already processed question: What is CCo?
Skipping already processed question: What is the difference between Data SAP and Data link control SAP?
Skipping already processed question: What is the difference between basic signaling and high-level communication?
Skipping already processed question: What is the difference between MTC and PTC?
Skipping already processed question: What is ATS?
Skipping already processed question: What is TSS?
Skipping already processed question: What is EIM?
Skipping already processed question: What is CM_SLAC_PARAM.REQ?
Skipping already processed question: What is CM_SLAC_PARAM.CNF?
Skipping already processed question: What is CM_START_ATTEN_CHAR.IND?
Skipping already processed question: What is CM_MN

In [60]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams
from deepeval import evaluate
def load_json(path: str):
        with open(path, "r") as f:
            return json.load(f)
    
def write_json(path: str, data: dict):
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

def evaluate_correctness_with_checkpoint(ground_truth_QA_path, generated_answers_path, eval_res_correctness_path):
        """
        Evaluates the correctness of LLM outputs against expected outputs with checkpointing.

        Parameters:
        - ground_truth_QA_path (str): Path to the ground truth QA JSON file.
        - generated_answers_path (str): Path to the generated answers JSON file.
        - eval_res_path (str): Path to the evaluation results JSON file.
        """
        ground_truth_QA = load_json(ground_truth_QA_path)
        generated_answers = load_json(generated_answers_path)
        if not ground_truth_QA or not generated_answers:
            print("No matched QA data to process.")
            return
        
        # Load existing evaluation results to avoid reprocessing
        eval_results = load_json(eval_res_correctness_path)
        processed_questions = set([result['question'] for result in eval_results])
        # Initialize the correctness metric
        correctness_metric = GEval(
            name="Correctness",
            criteria="Determine whether the actual output is factually correct based on the expected output.",
            #NOTE: you can only provide either criteria or evaluation_steps, and not both
            evaluation_steps=[
                "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
                "You should also heavily penalize omission of detail",
                "Vague language, or contradicting OPINIONS, are OK"
            ],
            # criteria="Determine whether the actual output is factually correct and complete based on the expected output. Additional relevant detail beyond the expected output should be considered a positive attribute, not a negative one.",
            # evaluation_steps=[
            #     "Check whether the facts in 'actual output' contradict any facts in 'expected output'. Factual contradictions should be severely penalized.",
            #     "Additional detail in the actual output beyond what's in the expected output should be considered a positive attribute, not a negative one, as long as the additional information is accurate and relevant.",
            #     "Evaluate the core information from the expected output - is it all present in the actual output? Missing key information should be penalized.",
            #     "Consider whether the additional detail enhances understanding or provides useful context - this should be rewarded.",
            #     "Vague language or contradicting opinions are acceptable as long as they don't misrepresent factual information."
            # ],
            evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
            #model=azure_openai,  # Replace with your actual model instance if different
            model="gpt-4.1",
            # model="o1",
            async_mode=False
        )
        
        total = len(generated_answers)
        print(f"Total entries to process: {total}")
        processed = len(processed_questions)
        print(f"Already processed entries: {processed}")
        res = []
        idx = 0
        try:
            for question, answer in generated_answers.items():
                print(question)
                expected_output = ground_truth_QA.get(question)
                idx += 1
                if not question:
                    print(f"Entry {idx} has no question. Skipping.")
                    continue
                if not expected_output:
                    print(f"Entry {idx} has no expected_output. Skipping.")
                    continue
                if not answer or answer.lower() == "null":
                    print(f"Entry {idx} has no valid answer. Skipping.")
                    continue
                if question in processed_questions:
                    print(f"Entry {idx}: '{question}' already processed. Skipping.")
                    continue
                
                # Create the test case
                test_case = LLMTestCase(
                    input=question,
                    actual_output=answer,
                    expected_output=expected_output
                )

                # Evaluate the test case
                evaluation_result = evaluate([test_case], [correctness_metric])
                if evaluation_result and evaluation_result.test_results:
                    test_result = evaluation_result.test_results[0]  # Get first test result
                    if test_result.metrics_data:
                        metric_data = test_result.metrics_data[0]  # Get first metric data
                    else:
                        print(f"No metrics data available for entry {idx}")
                else:
                    print(f"No evaluation results for entry {idx}")
                    
                # evaluate([test_case], [correctness_metric])
                # result = evaluate([test_case], [correctness_metric])
                # metric_data = result.test_result[0].metrics_data[0]
                result_entry = {
                    "question": question,
                    "score": metric_data.score,
                    "reason": metric_data.reason
                }
                res.append(result_entry)
                # Save the result incrementally
                write_json(eval_res_correctness_path, res)
                
                # Update processed questions
                processed_questions.add(question)
                
                print(f"Entry {idx}/{total} evaluated and saved.")
        
        except KeyboardInterrupt:
            print("\nEvaluation interrupted by user. Progress saved to eval_res.json.")

        except Exception as e:
            print(f"\nAn error occurred during evaluation: {e}")
            print("Progress saved up to the last successfully processed entry.")

        finally:
            print(f"\nEvaluation completed. Results saved to {eval_res_correctness_path}.")


In [ ]:
evaluate_correctness_with_checkpoint("../Answers/o4mini_golden_ans.json", "./fixed_entities_chunks_qa.json", "./fixed_correctness.json")
evaluate_correctness_with_checkpoint("../Answers/o4mini_golden_ans.json", "./dynamic_entities_chunks_qa.json", "./dynamic_correctness.json")

In [97]:
fixed_correctness = json.load(open("./fixed_correctness.json"))
dynamic_correctness = json.load(open("./dynamic_correctness.json"))
print(f"fixed_correctness: {sum([eval_res['score'] for eval_res in fixed_correctness]) / len(fixed_correctness)}")
print(f"dynamic_correctness: {sum([eval_res['score'] for eval_res in dynamic_correctness]) / len(dynamic_correctness)}")
        

fixed_correctness: 0.7594487388103677
dynamic_correctness: 0.8042143127302598


In [22]:
fixed_correctness = json.load(open("./fixed_correctness.json"))
dynamic_correctness = json.load(open("./dynamic_correctness.json"))
print(f"fixed_correctness: {sum([eval_res['score'] for eval_res in fixed_correctness]) / len(fixed_correctness)}")
print(f"dynamic_correctness: {sum([eval_res['score'] for eval_res in dynamic_correctness]) / len(dynamic_correctness)}")
        

fixed_correctness: 0.6800576076746064
dynamic_correctness: 0.6126083187356325


In [90]:
compress_prompt = load_json("./compressed_prompt.json")
output_file = "./original_qa.json"
qa_dict = load_json(output_file)
total_tokens = 0
for question, data in compress_prompt.items():
        # Skip if question has already been processed
        if question in qa_dict:
            print(f"Skipping already processed question: {question}")
            continue
        prompt = data['compressed_prompt']
        response = llm.invoke(prompt)
        qa_dict[question] = response.content
        print(response)
        total_tokens += response.usage_metadata['total_tokens']
        # Save after each question to preserve progress
        with open(output_file, "w") as f:
            json.dump(qa_dict, f, indent=4)

content='The Vehicle-to-Grid (V2G) system architecture is a complex framework that facilitates bidirectional communication and energy exchange between electric vehicles (EVs) and the power grid. This architecture is designed to optimize energy use, enhance grid stability, and enable electric vehicles to act as energy storage units. Below is a detailed breakdown of the components and interactions within the V2G system architecture:\n\n### 1. **Key Components**\n\n#### a. **Electric Vehicles (EVs)**\n- **Description**: EVs are the primary participants in the V2G system. They use electric motors for propulsion and are equipped with communication capabilities to interact with the grid and charging infrastructure.\n- **Functionality**: EVs can charge from the grid and return electricity back to the grid, helping to balance supply and demand.\n\n#### b. **Electric Vehicle Communication Controller (EVCC)**\n- **Description**: The EVCC is a critical component that manages communication between

In [93]:
print(total_tokens)

273444


In [ ]:
evaluate_correctness_with_checkpoint("../Answers/o4mini_golden_ans.json", "./original_qa.json", "./original_correctness.json")


In [92]:
original_correctness = json.load(open("./original_correctness.json"))
print(f"original_correctness: {sum([eval_res['score'] for eval_res in original_correctness]) / len(original_correctness)}")

original_correctness: 0.7242390010644145


In [99]:
compress_prompt = load_json("./final_prompt.json")
output_file = "./non_compressed_ans.json"
qa_dict = load_json(output_file)
total_tokens = 0
for question, prompt in compress_prompt.items():
        # Skip if question has already been processed
        if question in qa_dict:
            print(f"Skipping already processed question: {question}")
            continue
        response = llm.invoke(prompt)
        qa_dict[question] = response.content
        print(response)
        total_tokens += response.usage_metadata['total_tokens']
        # Save after each question to preserve progress
        with open(output_file, "w") as f:
            json.dump(qa_dict, f, indent=4)

content='The Vehicle-to-Grid (V2G) system architecture is a complex framework that facilitates bidirectional communication and energy exchange between electric vehicles (EVs) and the power grid. This architecture is built upon several key components and standards that ensure effective communication, control, and management of energy resources. Below is a detailed breakdown of the system architecture of V2G:\n\n### 1. **Core Components**\n   - **Electric Vehicle (EV)**: The EV is a crucial component that uses electric motors for propulsion and can communicate with the grid. It is equipped with communication modules that allow it to send and receive messages related to charging and energy exchange.\n   - **Electric Vehicle Communication Controller (EVCC)**: This component manages communication between the EV and the Electric Vehicle Supply Equipment (EVSE). It acts as a client in the communication protocol and is responsible for executing test cases and managing the validation processes.

In [100]:
print(total_tokens)

339591


In [ ]:
evaluate_correctness_with_checkpoint("../Answers/o4mini_golden_ans.json", "./non_compressed_ans.json", "./non_compressed_correctness.json")


In [102]:
non_compressed_correctness = json.load(open("./non_compressed_correctness.json"))
print(f"non_cmp_correctness: {sum([eval_res['score'] for eval_res in non_compressed_correctness]) / len(non_compressed_correctness)}")

non_cmp_correctness: 0.8236905609269984
